In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter

%matplotlib qt
from tensorflow.keras.datasets import cifar10

# import module with layers
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model_module")

from adaptive_layer import *

In [2]:
tf.random.set_seed(5)
np.random.seed(5)

In [2]:
def illustrate_train(name, y_label, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    epochs = range(1, len(static_f)+1)
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    plt.title(name, fontsize="xx-large")

    ax.set_ylabel(y_label, fontsize="large")
    ax.set_xlabel('Epochs', fontsize="large")

    ax.plot(epochs, static_f, label='ReLU Function')
    ax.plot(epochs, adaptive_f, label='Swish Function')
    ax.plot(epochs, adaptive_nn, label='Adaptive NN')
    ax.plot(epochs, adaptive_conv_nn, label='Adaptive Conv NN')

    ax.legend()
    
    plt.grid()
    plt.show()

In [3]:
def illustrate_test(name, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    
    x = ["ReLU Function", "Swish Function", "Adaptive NN", "Adaptive Conv NN"]
    y = [static_f, adaptive_f, adaptive_nn, adaptive_conv_nn]
    
    plt.title(name, fontsize="xx-large")
    ax.set_ylabel("Accuracy", fontsize="large")
    
    ax.bar(x, y)
    
    plt.grid()
    plt.show()

In [4]:
workbook = xlsxwriter.Workbook('cifar10_pretrained_measurements.xlsx')

In [6]:
def save_data(name, static_f_train, adaptive_f_train, adaptive_nn_train, adaptive_conv_nn_train, 
              static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test):
    
    epochs = len(static_f_train[0][:])
    worksheet = workbook.add_worksheet(name)
    
    worksheet.write(1, 1, name + " Training Loss")
    worksheet.write(1, 7, name + " Training Accuracy")
    worksheet.write(1, 13, name + " Test Accuracy")
    
    worksheet.write(2, 1, "Epochs")
    worksheet.write(2, 2, "ReLU Function")
    worksheet.write(2, 3, "Swish Function")
    worksheet.write(2, 4, "Adaptive NN")
    worksheet.write(2, 5, "Adaptive Conv NN")
    
    worksheet.write(2, 7, "Epochs")
    worksheet.write(2, 8, "ReLU Function")
    worksheet.write(2, 9, "Swish Function")
    worksheet.write(2, 10, "Adaptive NN")
    worksheet.write(2, 11, "Adaptive Conv NN")
    
    worksheet.write(2, 13, "Static Function")
    worksheet.write(2, 14, "ReLU Function")
    worksheet.write(2, 15, "Swish NN")
    worksheet.write(2, 16, "Adaptive Conv NN")
    
    worksheet.write(3, 13, static_f_test)
    worksheet.write(3, 14, adaptive_f_test)
    worksheet.write(3, 15, adaptive_nn_test)
    worksheet.write(3, 16, adaptive_conv_nn_test)
    
    for i in range(0, epochs):
        worksheet.write(3+i, 1, i+1)
        worksheet.write(3+i, 2, static_f_train[0][i])
        worksheet.write(3+i, 3, adaptive_f_train[0][i])
        worksheet.write(3+i, 4, adaptive_nn_train[0][i])
        worksheet.write(3+i, 5, adaptive_conv_nn_train[0][i])
        
        worksheet.write(3+i, 7, i+1)
        worksheet.write(3+i, 8, static_f_train[1][i])
        worksheet.write(3+i, 9, adaptive_f_train[1][i])
        worksheet.write(3+i, 10, adaptive_nn_train[1][i])
        worksheet.write(3+i, 11, adaptive_conv_nn_train[1][i])
    
    workbook.close()

In [7]:
# dataset

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train/255
x_test = x_test/255
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [8]:
EPOCHS = 5
name = "Cifar10 Pretrained Conv"
pretrained_model = keras.models.load_model("pretrained_m")
pretrained_model.trainable = False

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True)

flag = measurements = 1

static_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_nn_hist = np.zeros(shape=[2, EPOCHS])
adaptive_conv_nn_hist = np.zeros(shape=[2, EPOCHS])

static_f_test = 0
adaptive_f_test = 0
adaptive_nn_test = 0
adaptive_conv_nn_test = 0

In [9]:
pretrained_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 64)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 8, 128)         0         
 g2D)                                                   

In [10]:
static_f_model = keras.Sequential([
    pretrained_model,
    keras.layers.Dense(1000, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])

In [11]:
adaptive_f_model = keras.Sequential([
    pretrained_model,
    keras.layers.Dense(1000, activation=tf.nn.swish),
    keras.layers.Dense(10, activation="softmax"),
])

In [12]:
# Adaptive Layer
adaptive_nn_model = keras.Sequential([
    pretrained_model,
    AdaptiveLayer(1000, structure=[3, 2, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.leaky_relu, skip_w=0.3), 
    AdaptiveLayer(10, structure=[5, 2, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.softmax, skip_w=0.2)
])

In [13]:
# Adaptive Layer based on 1D Conv
adaptive_conv_nn_model = AdaptiveModel([
    pretrained_model,
    AdaptiveLayerConv(1000, structure=[24, 12], split=1, noise=0, skip_w=0.9, 
                      inner_hidden_activation=tf.nn.leaky_relu, inner_out_activation=tf.nn.leaky_relu), 
    AdaptiveLayerConv(10, structure=[32, 16, 8], split=2, noise=0, skip_w=0.9, 
                      inner_hidden_activation=tf.nn.leaky_relu, inner_out_activation=tf.nn.softmax) 
])

In [14]:
static_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=lr_schedule), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [15]:
adaptive_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=lr_schedule), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [16]:
adaptive_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=lr_schedule), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [17]:
adaptive_conv_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=lr_schedule), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [18]:
hist = static_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/5
782/782 [==============================] - 57s 73ms/step - loss: 0.3696 - accuracy: 0.8712
Epoch 2/5
782/782 [==============================] - 64s 81ms/step - loss: 0.1384 - accuracy: 0.9510
Epoch 3/5
782/782 [==============================] - 67s 85ms/step - loss: 0.0651 - accuracy: 0.9778
Epoch 4/5
782/782 [==============================] - 69s 88ms/step - loss: 0.0387 - accuracy: 0.9880
Epoch 5/5
782/782 [==============================] - 71s 90ms/step - loss: 0.0247 - accuracy: 0.9925


In [19]:
static_f_hist[0][:] += np.array(hist.history["loss"])/measurements
static_f_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [20]:
hist = adaptive_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/5
782/782 [==============================] - 65s 83ms/step - loss: 0.3675 - accuracy: 0.8712
Epoch 2/5
782/782 [==============================] - 70s 90ms/step - loss: 0.1338 - accuracy: 0.9537
Epoch 3/5
782/782 [==============================] - 71s 91ms/step - loss: 0.0588 - accuracy: 0.9800
Epoch 4/5
782/782 [==============================] - 72s 92ms/step - loss: 0.0322 - accuracy: 0.9898
Epoch 5/5
782/782 [==============================] - 72s 92ms/step - loss: 0.0225 - accuracy: 0.9927


In [21]:
adaptive_f_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_f_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [22]:
hist = adaptive_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/5
782/782 [==============================] - 59s 75ms/step - loss: 0.5964 - accuracy: 0.7889
Epoch 2/5
782/782 [==============================] - 66s 84ms/step - loss: 0.2075 - accuracy: 0.9342
Epoch 3/5
782/782 [==============================] - 69s 88ms/step - loss: 0.1070 - accuracy: 0.9635
Epoch 4/5
782/782 [==============================] - 70s 90ms/step - loss: 0.0541 - accuracy: 0.9828
Epoch 5/5
782/782 [==============================] - 71s 91ms/step - loss: 0.0288 - accuracy: 0.9913


In [23]:
adaptive_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_nn_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [24]:
hist = adaptive_conv_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/5
782/782 [==============================] - 63s 80ms/step - loss: 0.3816 - accuracy: 0.8675
Epoch 2/5
782/782 [==============================] - 69s 88ms/step - loss: 0.1518 - accuracy: 0.9463
Epoch 3/5
782/782 [==============================] - 72s 92ms/step - loss: 0.0916 - accuracy: 0.9680
Epoch 4/5
782/782 [==============================] - 73s 93ms/step - loss: 0.0568 - accuracy: 0.9808
Epoch 5/5
782/782 [==============================] - 74s 95ms/step - loss: 0.0375 - accuracy: 0.9879


In [25]:
adaptive_conv_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_conv_nn_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [26]:
static_f_test += static_f_model.evaluate(x_test, y_test)[1]/measurements

313/313 [==============================] - 13s 42ms/step - loss: 1.4376 - accuracy: 0.7908


In [27]:
adaptive_f_test += adaptive_f_model.evaluate(x_test, y_test)[1]/measurements

313/313 [==============================] - 13s 40ms/step - loss: 1.4759 - accuracy: 0.7874


In [28]:
adaptive_nn_test += adaptive_nn_model.evaluate(x_test, y_test)[1]/measurements

313/313 [==============================] - 13s 41ms/step - loss: 1.3946 - accuracy: 0.7845


In [29]:
adaptive_conv_nn_test += adaptive_conv_nn_model.evaluate(x_test, y_test)[1]/measurements
flag -= 1
if(flag == 0):
    print("---------FINISH---------")

313/313 [==============================] - 13s 41ms/step - loss: 1.5245 - accuracy: 0.7804
---------FINISH---------


In [30]:
print("Static Function model\nTest accuracy : ", static_f_test)
print("\nAdaptive Function model\nTest accuracy : ", adaptive_f_test)
print("\nAdaptive NN model\nTest accuracy : ", adaptive_nn_test)
print("\nAdaptive Conv NN model\nTest accuracy : ", adaptive_conv_nn_test)

Static Function model
Test accuracy :  0.7907999753952026

Adaptive Function model
Test accuracy :  0.7874000072479248

Adaptive NN model
Test accuracy :  0.784500002861023

Adaptive Conv NN model
Test accuracy :  0.7803999781608582


In [31]:
illustrate_train(name+" Training Loss", "Loss", static_f_hist[0][:], 
                 adaptive_f_hist[0][:], adaptive_nn_hist[0][:], adaptive_conv_nn_hist[0][:])

In [32]:
illustrate_train(name + " Training Accuracy", "Accuracy", static_f_hist[1][:], 
                 adaptive_f_hist[1][:], adaptive_nn_hist[1][:], adaptive_conv_nn_hist[1][:])

In [33]:
illustrate_test(name + " Test Accuracy", 
                static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)

In [34]:
save_data(name, static_f_hist, adaptive_f_hist, adaptive_nn_hist, 
          adaptive_conv_nn_hist, static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)